# Exploração de GANs para Transferência de Estilo



## 1. Introdução
Este projeto pretende explorar Redes Adversárias Generativas (GANs) para a transferência de estilo de imagens. O objetivo é transformar imagens de Pokémon no estilo de pinturas de artistas famosos, como Van Gogh. Este trabalho baseia-se em métodos como CycleGAN.
Este projeto usou NVIDIA CUDA, através do PC, sem recurso a Google Colab. A gráfica utilizada foi uma NVIDIA GEFORCE GTX 1650 ti.

## 2. Imports necessários
Uma vez que nas aulas usamos pyTorch, o trabalho foi desenvolvido usando essa biblioteca.


In [ ]:
import os
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

## 3. Preparação do Dataset
Nesta secção, descreve-se a preparação dos datasets, que incluem imagens de Pokémon e pinturas de Van Gogh.

### 3.1 Diretórios das Imagens
Definem-se os caminhos para os diretórios onde se encontram as imagens de Pokémon e Van Gogh.

In [ ]:
pokemon_directory = 'C:/Users/ferna/OneDrive/Ambiente de Trabalho/4 ano 2 sem/CG/Visão por Computador e Processamento Imagem/Visão por computador/VCPI_Individual/Pokemon/pokemon/pokemon/all_images'
van_gogh_directory = 'C:/Users/ferna/OneDrive/Ambiente de Trabalho/4 ano 2 sem/CG/Visão por Computador e Processamento Imagem/Visão por computador/VCPI_Individual/VincentVanGogh/VincentVanGogh/Saint Remy'

class ImageDataset(Dataset):
    def __init__(self, root_pokemon, root_painter, transform=None, limit=None):
        self.transform = transform
        self.files_pokemon = [os.path.join(root_pokemon, f) for f in os.listdir(root_pokemon) if f.endswith(('.png', '.jpg'))][:limit]
        self.files_painter = [os.path.join(root_painter, f) for f in os.listdir(root_painter) if f.endswith(('.png', '.jpg'))]

    def __getitem__(self, index):
        item_pokemon = self.transform(Image.open(self.files_pokemon[index % len(self.files_pokemon)]).convert('RGB'))
        item_painter = self.transform(Image.open(self.files_painter[index % len(self.files_painter)]).convert('RGB'))
        return {'A': item_pokemon, 'B': item_painter}

    def __len__(self):
        return max(len(self.files_pokemon), len(self.files_painter))

# Limitar a 100 imagens de Pokémon
pokemon_limit = 100
dataloader_van_gogh = DataLoader(ImageDataset(pokemon_directory, van_gogh_directory, transform=transform, limit=pokemon_limit), batch_size=1, shuffle=True, num_workers=0)
dataloader_monet = DataLoader(ImageDataset(pokemon_directory, monet_directory, transform=transform, limit=pokemon_limit), batch_size=1, shuffle=True, num_workers=0)

# Visualizar exemplos
for i, batch in enumerate(itertools.islice(dataloader_van_gogh, 3)):
	plt.figure(figsize=(10, 5))
	plt.subplot(1, 2, 1)
	plt.title('Van Gogh')
	plt.imshow(batch['B'][0].permute(1, 2, 0))
	plt.axis('off')
	plt.subplot(1, 2, 2)
	plt.title('Pokémon')
	plt.imshow(batch['A'][0].permute(1, 2, 0))
	plt.axis('off')
	plt.show()

### 3.2 Transformações e Augmentação de Dados
As imagens são redimensionadas, transformadas em tensor e normalizadas. Adicionalmente, aplica-se um flip horizontal aleatório para aumentar a variedade dos dados de treino.

In [ ]:
# Transformações com Data Augmentation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

### 3.3 Classe de Dataset Personalizada
Define-se uma classe de dataset personalizada para carregar as imagens de Pokémon e Van Gogh, limitando o número de imagens de Pokémon a 100.

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root_pokemon, root_painter, transform=None, limit=None):
        self.transform = transform
        self.files_pokemon = [os.path.join(root_pokemon, f) for f in os.listdir(root_pokemon) if f.endswith(('.png', '.jpg'))][:limit]
        self.files_painter = [os.path.join(root_painter, f) for f in os.listdir(root_painter) if f.endswith(('.png', '.jpg'))]

    def __getitem__(self, index):
        item_pokemon = self.transform(Image.open(self.files_pokemon[index % len(self.files_pokemon)]).convert('RGB'))
        item_painter = self.transform(Image.open(self.files_painter[index % len(self.files_painter)]).convert('RGB'))
        return {'A': item_pokemon, 'B': item_painter}

    def __len__(self):
        return max(len(self.files_pokemon), len(self.files_painter))

### 3.4 Criação dos DataLoaders
Os DataLoaders são criados para carregar as imagens durante o treino, garantindo que são embaralhadas.

In [ ]:
pokemon_limit = 100
dataloader = DataLoader(ImageDataset(pokemon_directory, van_gogh_directory, transform=transform, limit=pokemon_limit), batch_size=1, shuffle=True, num_workers=0)

## 4. Arquitetura dos modelos
Nesta secção, detalha-se a arquitetura das redes geradoras e discriminadoras utilizadas.

### 4.1 Gerador
O gerador transforma uma imagem de um domínio para outro, neste caso, de Pokémon para um estilo de pintura. A arquitetura do gerador é composta por uma camada inicial de convolução, seguida por camadas de downsampling, blocos residuais, camadas de upsampling e uma camada de saída.

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_nc, output_nc):
        super(Generator, self).__init__()

        # Bloco de convolução inicial
        model = [
            nn.Conv2d(input_nc, 64, kernel_size=7, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        ]

        # Downsampling
        in_features = 64
        out_features = in_features * 2
        for _ in range(2):
            model += [
                nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1, bias=False),
                nn.BatchNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features * 2

        # Blocos residuais
        for _ in range(6):
            model += [
                nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(in_features),
                nn.ReLU(inplace=True)
            ]

        # Upsampling
        out_features = in_features // 2
        for _ in range(2):
            model += [
                nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False),
                nn.BatchNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features // 2

        # Camada de saída
        model += [nn.Conv2d(64, output_nc, kernel_size=7, padding=3), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

### 4.2 Discriminador
O discriminador distingue entre imagens reais e geradas. A arquitetura do discriminador é composta por várias camadas de convolução que reduzem progressivamente a dimensão da imagem, até uma camada final que produz um mapa de probabilidade indicando se as regiões da imagem são reais ou falsas.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()
        model = [
            nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
        ]
        in_features = 64
        out_features = in_features * 2
        for _ in range(3):
            model += [
                nn.Conv2d(in_features, out_features, kernel_size=4, stride=2, padding=1, bias=False),
                nn.BatchNorm2d(out_features),
                nn.LeakyReLU(0.2, inplace=True)
            ]
            in_features = out_features
            out_features = in_features * 2
        model += [nn.Conv2d(in_features, 1, kernel_size=4, stride=1, padding=1, bias=False)]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


## 5. Inicialização e Treino do Modelo
Nesta secção, inicializam-se e treinam-se os modelos.

### 5.1 Inicialização dos Pesos
Inicializam-se os pesos das redes para garantir uma distribuição normal. Esta técnica é importante para assegurar que os pesos começam com valores que permitem ao modelo aprender de forma eficiente.

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

### 5.2 Definição das Funções de Perda
Utilizam-se três funções de perda diferentes:

- criterion_GAN: Mede a diferença entre as previsões do discriminador para imagens reais e geradas, usando Mean Squared Error (MSE).
- criterion_cycle: Calcula a diferença entre a imagem original e a imagem reconstruída após passar pelo gerador de ida e volta, utilizando L1 Loss. Esta perda garante que a imagem reconstruída seja semelhante à original.
- criterion_identity: Verifica se o gerador mantém a identidade da imagem quando a imagem de destino é fornecida como entrada, utilizando L1 Loss.

In [ ]:
criterion_GAN = nn.MSELoss().to(device)
criterion_cycle = nn.L1Loss().to(device)
criterion_identity = nn.L1Loss().to(device)

### 5.3 Inicialização dos Modelos e Otimizadores
Inicializam-se os modelos e otimizadores necessários para o treino.

In [ ]:
netG_A2B = Generator(input_nc=3, output_nc=3).to(device)
netG_B2A = Generator(input_nc=3, output_nc=3).to(device)
netD_A = Discriminator(input_nc=3).to(device)
netD_B = Discriminator(input_nc=3).to(device)

netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)

optimizer_G = optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_A = optim.Adam(netD_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = optim.Adam(netD_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

scheduler_G = optim.lr_scheduler.StepLR(optimizer_G, step_size=100, gamma=0.5)
scheduler_D_A = optim.lr_scheduler.StepLR(optimizer_D_A, step_size=100, gamma=0.5)
scheduler_D_B = optim.lr_scheduler.StepLR(optimizer_D_B, step_size=100, gamma=0.5)

## 6. Ciclo de Treino
Nesta secção, descreve-se o ciclo de treino do modelo. Cada época envolve os seguintes passos:

1. Geradores: Atualizam-se os geradores para gerar imagens que enganam os discriminadores.

- Identidade: A imagem gerada a partir da imagem de destino deve ser igual à própria imagem de destino.
- Adversarial: A imagem gerada deve ser classificada como real pelo discriminador.
- Ciclo: A imagem original deve ser recuperada após passar pelos dois geradores (ciclo de ida e volta).

2. Discriminadores: Atualizam-se os discriminadores para distinguir entre imagens reais e geradas.

- A imagem real deve ser classificada como real.
- A imagem gerada deve ser classificada como falsa.
3. Atualização dos Otimizadores: Os otimizadores são atualizados após o cálculo das perdas.

In [ ]:
def train_epoch(epoch, dataloader, netG_A2B, netG_B2A, netD_A, netD_B, optimizer_G, optimizer_D_A, optimizer_D_B, criterion_GAN, criterion_cycle, criterion_identity, device):
    for i, batch in enumerate(dataloader):
        real_A = batch['A'].to(device)
        real_B = batch['B'].to(device)

        valid = torch.ones((real_A.size(0), *netD_A(real_A).shape[1:]), requires_grad=False).to(device)
        fake = torch.zeros((real_A.size(0), *netD_A(real_A).shape[1:]), requires_grad=False).to(device)

        optimizer_G.zero_grad()

        loss_id_A = criterion_identity(netG_B2A(real_A), real_A) * 5.0
        loss_id_B = criterion_identity(netG_A2B(real_B), real_B) * 5.0

        fake_B = netG_A2B(real_A)
        loss_GAN_A2B = criterion_GAN(netD_B(fake_B), valid)
        fake_A = netG_B2A(real_B)
        loss_GAN_B2A = criterion_GAN(netD_A(fake_A), valid)

        rec_A = netG_B2A(fake_B)
        loss_cycle_A = criterion_cycle(rec_A, real_A) * 10.0
        rec_B = netG_A2B(fake_A)
        loss_cycle_B = criterion_cycle(rec_B, real_B) * 10.0

        loss_G = loss_id_A + loss_id_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_A + loss_cycle_B
        loss_G.backward()
        optimizer_G.step()

        optimizer_D_A.zero_grad()
        loss_real_A = criterion_GAN(netD_A(real_A), valid)
        loss_fake_A = criterion_GAN(netD_A(fake_A.detach()), fake)
        loss_D_A = (loss_real_A + loss_fake_A) * 0.5
        loss_D_A.backward()
        optimizer_D_A.step()

        optimizer_D_B.zero_grad()
        loss_real_B = criterion_GAN(netD_B(real_B), valid)
        loss_fake_B = criterion_GAN(netD_B(fake_B.detach()), fake)
        loss_D_B = (loss_real_B + loss_fake_B) * 0.5
        loss_D_B.backward()
        optimizer_D_B.step()

        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], '
                  f'Loss D_A: {loss_D_A.item():.4f}, Loss D_B: {loss_D_B.item():.4f}, '
                  f'Loss G: {loss_G.item():.4f}')

    scheduler_G.step()
    scheduler_D_A.step()
    scheduler_D_B.step()


### 6.1 Treino Principal
Treina-se o modelo ao longo de várias épocas utilizando o ciclo de treino definido.

In [ ]:
num_epochs = 100
dataloader = DataLoader(ImageDataset(pokemon_directory, van_gogh_directory, transform=transform, limit=pokemon_limit), batch_size=1, shuffle=True, num_workers=0)

for epoch in range(num_epochs):
    train_epoch(epoch, dataloader, netG_A2B, netG_B2A, netD_A, netD_B, optimizer_G, optimizer_D_A, optimizer_D_B, criterion_GAN, criterion_cycle, criterion_identity, device)